In [42]:
%matplotlib inline

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dicom
import os
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
import glob
import re
import xmltodict
import pickle

from skimage import measure, morphology
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.segmentation import clear_border
from skimage.filters import roberts, sobel

from mpl_toolkits.mplot3d.art3d import Poly3DCollection

DATA_PATH = '/kaggle_2/lidc_idri/data/'
DATA_PATH_XML = '/kaggle_2/lidc_idri/data/tcia-lidc-xml/'
DATA_PATH_SCANS = '/kaggle_2/lidc_idri/data/LIDC/DOI/'

In [ ]:
patient_ids = set()
paths = set()
non_ct_files = set()
for filename in glob.glob(DATA_PATH_XML + '**/*.xml', recursive=True):
    with open(filename) as fd:
        doc = xmltodict.parse(fd.read())
        try:
            patient_ids.add(doc['LidcReadMessage']['ResponseHeader']['SeriesInstanceUid'])
            paths.add(doc['LidcReadMessage']['ResponseHeader']['StudyInstanceUID'] + '/' +
                      doc['LidcReadMessage']['ResponseHeader']['SeriesInstanceUid'])
        except KeyError:
            non_ct_files.add(filename)
            continue

In [38]:
patient_scans_map = {}
count = 0
for path in paths:
    [study_id, patient_id] = path.split('/')
    scan_files = glob.glob(DATA_PATH_SCANS + '**/' + path + '/*.dcm', recursive=True)
    xml_files = glob.glob(DATA_PATH_SCANS + '**/' + path + '/*.xml', recursive=True)
    patient_scans_map[patient_id] = {
        'scans': scan_files,
        'xml': xml_files
    }
    count = count + 1
    print(count, patient_id, len(scan_files), len(xml_files))

1 1.3.6.1.4.1.14519.5.2.1.6279.6001.237915456403882324748189195892 122 1
2 1.3.6.1.4.1.14519.5.2.1.6279.6001.199069398344356765037879821616 472 1
3 1.3.6.1.4.1.14519.5.2.1.6279.6001.202283133206014258077705539227 123 1
4 1.3.6.1.4.1.14519.5.2.1.6279.6001.207544473852086582434957174616 101 1
5 1.3.6.1.4.1.14519.5.2.1.6279.6001.121391737347333465796214915391 244 1
6 1.3.6.1.4.1.14519.5.2.1.6279.6001.340012777775661021262977442176 133 1
7 1.3.6.1.4.1.14519.5.2.1.6279.6001.725236073737175770730904408416 216 1
8 1.3.6.1.4.1.14519.5.2.1.6279.6001.297251044869095073091780740645 449 1
9 1.3.6.1.4.1.14519.5.2.1.6279.6001.814122498113547115932318256859 132 1
10 1.3.6.1.4.1.14519.5.2.1.6279.6001.822128649427327893802314908658 131 1
11 1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950022352718462251777 291 1
12 1.3.6.1.4.1.14519.5.2.1.6279.6001.147250707071097813243473865421 474 1
13 1.3.6.1.4.1.14519.5.2.1.6279.6001.226456162308124493341905600418 241 1
14 1.3.6.1.4.1.14519.5.2.1.6279.6001.1236543563

In [43]:
with open(DATA_PATH + 'patient_scans_map.pkl', 'wb') as f:
    pickle.dump(patient_scans_map, f, pickle.HIGHEST_PROTOCOL)

In [44]:
with open(DATA_PATH + 'patient_scans_map.pkl', 'rb') as f:
    scans_map_copy = pickle.load(f)

In [45]:
len(scans_map_copy)

1018